# Weekly Update of the COVID-19 Cases in Top Ten Zip Code Areas
### Prepared for NIH RADx-Up Project: Communities Fighting COVID
Center for Human Dynamics in the Mobile Age (HDMA) at San Diego State University <br>
Jessica Embury

## Import Statements

In [ ]:
import pandas as pd
import arcpy
import webbrowser
import json
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer

## User Variables

In [ ]:
###########################
###SET DATA THROUGH DATE###
###########################
most_recent_date = '02/20/2021'  # Change date daily using 'MM/DD/YYYY' format
date_range = '2/14/21 - 2/20/21'

# modified date
d = most_recent_date.replace('/','')

##########################
###SET PATHS IN AND OUT###
##########################
#PATHS IN
path_in = '../covid_data/percents/covid_percents_{}.csv'.format(d)

#PATHS OUT
path_out = './data/radx_covid_upload.csv'
path_out2 = '../covid_data/radx_updates/radx_covid_update_{}.csv'.format(d)
#path out for dash updates, feature layer overwrite
path_out3 = '../covid_data/radx_updates/radx_covid_sites_update_{}.csv'.format(d)
path_out4 = '../covid_data/radx_updates/radx_covid_sites_upload.csv'

################################
###ARCGIS DETAILS FOR UPDATES###
################################
layer_id = '94f1c49aeb324fff91d35f8ab86c553f'
map_id = '6978702efc634f508003453e434417da'
mobilemap_id = '06bec9354a214a7ea0e781368eca5cd9'
dash = 'https://experience.arcgis.com/experience/906d9ccaa0894762ae7bfa8aa46d1809'

## Data Processing - Pandas

In [ ]:
# create df using covid spreadsheet from most_recent_date
df = pd.read_csv(path_in)

df.head()

In [ ]:
# delete unwanted columns
del df['Daily Increased']
del df['Daily Change Rate*1000']
del df['percent_total']
del df['percent_daily']

df.head()

In [ ]:
# rename columns to match new schema
df = df.rename(columns = {'2018_population': 'Population 2018', 'Date': 'Date Range', 'Confirmed Cases': 'Accumulated Confirmed Cases', 
                         'Rate Per 100K': 'Accumulated Rate Per 100K', '7 Day Case Increase': '7-Day Confirmed Cases', 
                          '7 Days Rolling Change*1000': '7-Day Change Rate*1000'})

df['Date Range'] = date_range

df.head()

In [ ]:
# create columns, calculate values for daily average and weekly case rate per 100k residents
df['7-Day Case Average'] = round(df['7-Day Confirmed Cases']/7, 2)

df['7-Day Case Rate Per 100K'] = round(df['7-Day Confirmed Cases']/df['Population 2018']*100000, 2)

df.head()

In [ ]:
# save df to csv files
df.to_csv(path_out, index=False)
df.to_csv(path_out2, index=False)

## Data Processing - arcpy
Calculate number of County test sites in each zip code and in adjacent zip codes, add as columns to df

In [ ]:
# csv path for gdb table
csv_in = path_out

# Get and set current project and geodatabase info
arcpy.env.overwriteOutput = True
aprx = arcpy.mp.ArcGISProject('CURRENT')
defaultGeoDb = arcpy.env.workspace
aprx.defaultGeodatabase = defaultGeoDb
currentMap = aprx.activeMap

# set names for table and layer
tempStr = 'radx_covid_update_{}.csv'.format(d)
newMapName= 'lyr_' + tempStr
newTableName = 'tbl_' + tempStr

# convert CSV data into table and add to default database - look in database to see it was added
arcpy.TableToTable_conversion(csv_in,defaultGeoDb,newTableName)

In [ ]:
# join radx case data to zipcode polygon
covid_data_join = arcpy.management.AddJoin('zips_test_sites', 'ZIP', newTableName, 'Zipcode')

# output joined layer to gdb
arcpy.CopyFeatures_management(covid_data_join, 'C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/{}'.format(newMapName).replace('.csv',''))

In [ ]:
# remove join
arcpy.RemoveJoin_management('zips_test_sites')

In [ ]:
# spatial join to get number of test sites in each zip code
# Join_Count = number of tests sites in each zip code
zip_sites = arcpy.analysis.SpatialJoin('zips_test_sites', 'COVID19_Testing_Locations', 
                                       'C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/sites_in_zips_{}'.format(d))

In [ ]:
# get zip code neighbors - do not need to re-run each time
# zip_nbrs = arcpy.analysis.PolygonNeighbors('zips_test_sites', 'C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/zip_neighbors', 'ZIP')

In [ ]:
#join zip_nbrs to zip_sites to eventually get number of sites in neighboring zip codes for each zip code
nbrs_join = arcpy.management.AddJoin('sites_in_zips_{}'.format(most_recent_date.replace('/', '')), 'ZIP', 
                                     'zip_neighbors', 'nbr_ZIP')
# output joined layer to gdb
arcpy.CopyFeatures_management(nbrs_join, 'C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/zip_nbr_join_{}'.format(d))

In [ ]:
# remove join
arcpy.RemoveJoin_management('sites_in_zips_{}'.format(most_recent_date.replace('/', '')))

In [ ]:
# dissolve zip_nbr_join to get sum of sites in neighboring zips
arcpy.management.Dissolve('C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/zip_nbr_join_{}'.format(d), 
                          'C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/zip_nbr_diss_{}'.format(d), 
                          'zip_neighbors_src_ZIP', 'sites_in_zips_{}_Join_Count SUM'.format(d))

In [ ]:
# join zip_nbr_diss and sites_in_zips
sites_join_in_adj = arcpy.management.AddJoin('sites_in_zips_{}'.format(d), 'ZIP', 
                                             'zip_nbr_diss_{}'.format(d), 'zip_neighbors_src_ZIP')
# output joined layer to gdb
arcpy.CopyFeatures_management(sites_join_in_adj, 'C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/zip_sites_in_and_adj_{}'.format(d))

In [ ]:
# remove join
arcpy.RemoveJoin_management('sites_in_zips_{}'.format(d))

In [ ]:
# join zip_sites_in_and_adj and lyr_radx_covid_update
radx_sites_join = arcpy.management.AddJoin(newMapName, 'zips_test_sites_ZIP', 
                                             'zip_sites_in_and_adj_{}'.format(d), 
                                             'sites_in_zips_{}_ZIP'.format(d))

# output joined layer to gdb
arcpy.CopyFeatures_management(radx_sites_join, 'C:/Users/jesse/iCloudDrive/HDMA/covid_dashboards/Default.gdb/radx_7day_update_{}'.format(d))

In [ ]:
# remove join
arcpy.RemoveJoin_management(newMapName)

In [ ]:
# del unnecessary fields from radx_7day_update

# layer name
layer_name = 'radx_7day_update_{}'.format(d)

# repeat field name segments
f1 = 'lyr_radx_covid_update_{}'.format(d)
f2 = 'zip_sites_in_and_adj_{}'.format(d)
f3 = 'sites_in_zips_{}'.format(d)
f4 = '{}_tbl_radx_covid_update_{}'.format(f1, d)

arcpy.DeleteField_management(layer_name, 
                             ['{}_zips_test_sites_ZIP'.format(f1),
                             '{}_zips_test_sites_zip_text'.format(f1), 
                             '{}_OB'.format(f4),
                             '{}_OBJECTID'.format(f2),
                             '{}_{}_TARGET_FID'.format(f2, f3),
                             '{}_{}_ZIP'.format(f2, f3),
                             '{}_{}_zip_text'.format(f2, f3),
                             '{}_{}_facilityid'.format(f2, f3),
                             '{}_{}_name'.format(f2, f3),
                             '{}_{}_fulladdr'.format(f2, f3),
                             '{}_{}_municipalit'.format(f2, f3),
                             '{}_{}_agency'.format(f2, f3),
                             '{}_{}_agencytype'.format(f2, f3),
                             '{}_{}_phone'.format(f2, f3),
                             '{}_{}_agencyurl'.format(f2, f3),
                             '{}_{}_operhours'.format(f2, f3),
                             '{}_{}_comments'.format(f2, f3),
                             '{}_{}_Instruction'.format(f2, f3),
                             '{}_{}_status'.format(f2, f3),
                             '{}_{}_CreationDat'.format(f2, f3),
                             '{}_{}_EditDate'.format(f2, f3),
                             '{}_{}_drive_throu'.format(f2, f3),
                             '{}_{}_appt_only'.format(f2, f3),
                             '{}_{}_referral_re'.format(f2, f3),
                             '{}_{}_services_of'.format(f2, f3),
                             '{}_{}_call_first'.format(f2, f3),
                             '{}_{}_virtual_scr'.format(f2, f3),
                             '{}_{}_health_dept'.format(f2, f3),
                             '{}_{}_State'.format(f2, f3),
                             '{}_{}_data_source'.format(f2, f3),
                             '{}_{}_county'.format(f2, f3),
                             '{}_{}_red_flag'.format(f2, f3),
                             '{}_{}_vol_note'.format(f2, f3),
                             '{}_{}_public_form'.format(f2, f3),
                             '{}_{}_start_date'.format(f2, f3),
                             '{}_{}_end_date'.format(f2, f3),
                             '{}_{}_type_of_tes'.format(f2, f3),
                             '{}_{}_test_proces'.format(f2, f3),
                             '{}_{}_fine_print'.format(f2, f3),
                             '{}_{}_bos_distric'.format(f2, f3),
                             '{}_{}_DISPLAY_PUB'.format(f2, f3),
                             '{}_{}_name_spanis'.format(f2, f3),
                             '{}_{}_operhours_s'.format(f2, f3),
                             '{}_{}_instructi_1'.format(f2, f3),
                             '{}_{}_operhours_n'.format(f2, f3),
                             '{}_zip_nbr_diss_{}_OBJECTID'.format(f2, d),
                             '{}_zip_nbr_diss_{}_zip_neighbor'.format(f2, d)
                             ])

In [ ]:
# rename fields

# [old name, new name, alias]
col_names = [
         ['{}_Zi'.format(f4), 'zipcode', 'Zip Code'],
         ['{}_Co'.format(f4), 'community', 'Community'],
         ['{}_La'.format(f4), 'lat', 'Latitude'],
         ['{}_Lo'.format(f4), 'lon', 'Longitude'],
         ['{}_Po'.format(f4), 'population2018', 'Population 2018'],
         ['{}_Da'.format(f4), 'date_range', 'Date Range'],
         ['{}_Ac'.format(f4),'accum_cases', 'Accumulated Confirmed Cases'],
         ['{}__1'.format(f4), 'accum_rate_100k_res', 'Accumulated Rate Per 100K'],
         ['{}_F7'.format(f4),'change_rate_7day', '7-Day Change Rate*1000'],
         ['{}__2'.format(f4), 'confirmed_cases_7day', '7-Day Confirmed Cases'],
         ['{}__3'.format(f4),'case_avg_7day', '7-Day Case Average'],
         ['{}__4'.format(f4), 'case_100kres_7day', '7-Day Case Rate Per 100K'],
         ['{}_sites_in_zips_{}_Join_Count'.format(f2, d), 'sites_in_zip', 'County Test Sites in Zip'],
         ['{}_zip_nbr_diss_{}_SUM_sites_in'.format(f2, d),'sites_adj_zips', 'County Test Sites in Adj. Zips']
        ]

# rename columns with AlterField
for name in col_names:
    arcpy.management.AlterField(layer_name, name[0], name[1], name[2])

In [ ]:
# save as csv file
arcpy.conversion.TableToTable('radx_7day_update_{}'.format(d),
                              '../covid_data/radx_updates',
                              'radx_covid_sites_update{}.csv'.format(d)
                             )

In [ ]:
# format table for layer overwrite
df2 = pd.read_csv('../covid_data/radx_updates/radx_covid_sites_update{}.csv'.format(d))

del df2['OID_']
del df2 ['Shape_Length']
del df2['Shape_Area']

for name in col_names:
    df2 = df2.rename(columns = {name[1] : name[2]})
                                
df2 = df2.dropna(subset = ['Zip Code'])

df2['Zip Code'] = df2['Zip Code'].astype(int)
df2['Population 2018'] = df2['Population 2018'].astype(int)
df2['Accumulated Confirmed Cases'] = df2['Accumulated Confirmed Cases'].astype(int)
df2['7-Day Confirmed Cases'] = df2['7-Day Confirmed Cases'].astype(int)
df2['County Test Sites in Zip'] = df2['County Test Sites in Zip'].astype(int)
df2['County Test Sites in Adj. Zips'] = df2['County Test Sites in Adj. Zips'].astype(int)

# save df to csv files
df2.to_csv(path_out3, index=False)
df2.to_csv(path_out4, index=False)

print(len(df2))
df2.head()

## Update ArcGIS Online Feature Layer and Maps

In [ ]:
# connect to arcgis account
gis = GIS('pro')

In [ ]:
# functions
def get_map (map_id):
    '''
    GET MAP DATA FOR SYMBOLOGY CHANGES
    '''  
    m = gis.content.get(map_id)
    data = m.get_data()    
    #Include this line for JSON
    #print(json.dumps(data, indent=4, sort_keys=True))
    print(m)    
    return data
    
def update_map (map_id, data):
    '''
    UPDATE MAP TO SAVE CHANGES
    '''
    m = gis.content.get(map_id)    
    # Set the item_properties to include the desired update
    properties = {"text": json.dumps(data)}
    # 'Commit' the updates to the Item
    update = m.update(item_properties=properties)
    return update

In [ ]:
# feature layer overwrite
#get feature layer containing updated data for maps associated with the COVID-19 dashboards
layer = gis.content.get(layer_id)
layer

layer_collection = FeatureLayerCollection.fromitem(layer)

#call the overwrite() method which can be accessed using the manager property
layer_collection.manager.overwrite(path_out4)

In [ ]:
# get max values to update map symbology - rate100k [14], average [13], changerate [12]
max_rate100k = df2['7-Day Case Rate Per 100K'].max()
max_average = df2['7-Day Case Average'].max()
max_changerate = df2['7-Day Change Rate*1000'].max()

print(max_rate100k, max_average, max_changerate)

In [ ]:
# update desktop map
#get map data
data = get_map(map_id)

#adjust symbology for graduated points to reflect new max/min
#MAX rate 100k
data['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_rate100k.item()
data['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_rate100k.item()

#max average
data['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_average.item()
data['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_average.item()

#max change rate
data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_changerate.item()
data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_changerate.item()

#update map to save changes
update = update_map(map_id, data)
update

In [ ]:
# update mobile map
#get map data
data2 = get_map(mobilemap_id)

#adjust symbology for graduated points to reflect new max/min
#MAX rate 100k
data2['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_rate100k.item()
data2['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_rate100k.item()

#max average
data2['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_average.item()
data2['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_average.item()

#max change rate
data2['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_changerate.item()
data2['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_changerate.item()

#update map to save changes
update2 = update_map(mobilemap_id, data2)
update2

In [ ]:
#Open dashboard in browser
webbrowser.open(dash, new=2)

## Create Tables for Weekly Report

In [ ]:
# Table 1:  Highest 7-Day Case Rates Per 100,000 Resident
df2[['Zip Code', 'Community', '7-Day Case Rate Per 100K', 'County Test Sites in Zip', 'County Test Sites in Adj. Zips']].sort_values(['7-Day Case Rate Per 100K'], ascending=False).head(10)#.to_string(index=False)

In [ ]:
# Table 2:  Highest 7-Day Average Case Burdens
df2[['Zip Code', 'Community', '7-Day Case Average', 'County Test Sites in Zip', 'County Test Sites in Adj. Zips']].sort_values(['7-Day Case Average'], ascending=False).head(10)#.to_string(index=False))

In [ ]:
# Table 3:  Highest 7-Day Average Change Rate
df2[['Zip Code', 'Community', '7-Day Change Rate*1000', 'County Test Sites in Zip', 'County Test Sites in Adj. Zips']].sort_values(['7-Day Change Rate*1000'], ascending=False).head(10)#.to_string(index=False))